# Archivos usados

In [1]:
# Archivos que se cargaron para cada función

import pandas as pd

df_reviews = pd.read_parquet("reviews_limpio.parquet")
df_items = pd.read_parquet("items_limpia.parquet")
df_games = pd.read_parquet("games_limpio.parquet")

# PlayTimeGenre
Devuelve el año con mas horas jugadas para dicho género.

In [2]:
df_games.rename(columns={'id': 'item_id'}, inplace=True)

In [3]:
# Modificaciones a los dataframes 
playtiem_item = df_items.groupby('item_id')['playtime_forever'].sum().reset_index()
df_merge = pd.merge(df_games, playtiem_item, on='item_id', how='inner')
df_PlayTimeGenre = df_merge[["genres","año_lanzamiento","playtime_forever"]]
df_PlayTimeGenre =df_PlayTimeGenre.dropna()
df_PlayTimeGenre.to_parquet(r'df_PlayTimeGenre.parquet',index=False)

In [6]:
def PlayTimeGenre(genero : str):
    # Filtrar el DataFrame para obtener solo las filas que contienen el género específico
    filtro_genero = df_PlayTimeGenre['genres'].str.contains(genero, case=False, na=False)
    df_filtrado = df_PlayTimeGenre[filtro_genero]
    df_agrupado = df_filtrado.groupby('año_lanzamiento')['playtime_forever'].sum().reset_index()
    # Encontrar el año con la mayor suma de 'playtime_forever'
    df_agrupado['año_lanzamiento'] = df_agrupado['año_lanzamiento'].astype(int)
    anio_mayor_suma = df_agrupado.loc[df_agrupado['playtime_forever'].idxmax(), 'año_lanzamiento']
    respuesta = {f"Año de lanzamiento con más horas jugadas para Género el {genero}": anio_mayor_suma}
    return respuesta 

PlayTimeGenre("Indie")

{'Año de lanzamiento con más horas jugadas para Género el Indie': 2006}

# UseForGenre
Devuelve el usuario que mas horas ha acumulado para un género establecido y una lista de acumulación de horas jugadas por año.

In [8]:
df_aux = df_games[["item_id","genres","año_lanzamiento"]]
df_UserForGenre = pd.merge(df_items,df_aux, on = "item_id")
df_UserForGenre =df_UserForGenre[["user_id","genres","año_lanzamiento","playtime_forever"]]
def eliminar_outliers_iqr(df, columna):
    # Calcular el rango intercuartílico (IQR)
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    # Definir los límites superior e inferior para identificar outliers
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    # Filtrar el DataFrame para mantener solo los valores dentro de los límites
    df_filtrado = df[(df[columna] >= limite_inferior) & (df[columna] <= limite_superior)]
    return df_filtrado

# Ejemplo
# Suponiendo que 'df' es tu DataFrame y 'valor' es la columna que contiene los datos
df_UserForGenre = eliminar_outliers_iqr(df_UserForGenre, 'playtime_forever')
df_UserForGenre.to_parquet(r'df_UserForGenre.parquet',index=False)

In [9]:
def userforgenre(genero: str):
    # Filtrar el DataFrame para el género especificado
    filtro_genero = df_UserForGenre['genres'].str.contains(genero, case=False, na=False)
    df_filtrado = df_UserForGenre[filtro_genero]

    # Agrupar por 'user_id' y 'year', sumar las horas jugadas
    df_agrupado = df_filtrado.groupby(['user_id', 'año_lanzamiento'])['playtime_forever'].sum().reset_index()

    # Encontrar el usuario con la máxima suma de horas jugadas
    idx_max_playtime = df_agrupado['playtime_forever'].idxmax()
    usuario_max_playtime = df_agrupado.loc[idx_max_playtime, 'user_id']

    # Filtrar el DataFrame para el usuario con máxima suma de horas jugadas
    df_usuario = df_agrupado[df_agrupado['user_id'] == usuario_max_playtime]

    # Crear el formato "Horas jugadas"
    resultado_final = [{'Año': int(row['año_lanzamiento']), 'Horas': int(row['playtime_forever'])} for _, row in df_usuario.iterrows()]
    
    return {"Usuario con más horas jugadas para Género {}:".format(genero): usuario_max_playtime, "Horas jugadas": resultado_final}

# Ejemplo de uso
userforgenre('actIon')

{'Usuario con más horas jugadas para Género actIon:': 'phrostb',
 'Horas jugadas': [{'Año': 1988, 'Horas': 92},
  {'Año': 1989, 'Horas': 0},
  {'Año': 1990, 'Horas': 0},
  {'Año': 1991, 'Horas': 0},
  {'Año': 1992, 'Horas': 0},
  {'Año': 1993, 'Horas': 1384},
  {'Año': 1994, 'Horas': 623},
  {'Año': 1995, 'Horas': 931},
  {'Año': 1996, 'Horas': 589},
  {'Año': 1997, 'Horas': 1005},
  {'Año': 1998, 'Horas': 202},
  {'Año': 1999, 'Horas': 512},
  {'Año': 2000, 'Horas': 635},
  {'Año': 2001, 'Horas': 400},
  {'Año': 2002, 'Horas': 737},
  {'Año': 2003, 'Horas': 657},
  {'Año': 2004, 'Horas': 375},
  {'Año': 2005, 'Horas': 1121},
  {'Año': 2006, 'Horas': 1396},
  {'Año': 2007, 'Horas': 1283},
  {'Año': 2008, 'Horas': 331},
  {'Año': 2009, 'Horas': 3960},
  {'Año': 2010, 'Horas': 3944},
  {'Año': 2011, 'Horas': 5056},
  {'Año': 2012, 'Horas': 13119},
  {'Año': 2013, 'Horas': 24141},
  {'Año': 2014, 'Horas': 53481},
  {'Año': 2015, 'Horas': 124185},
  {'Año': 2016, 'Horas': 68425},
  {'Año':

# UserRecommend
Devuelve el top de los tres juegos más recomendados por usuarios para un cierto año

In [14]:
# Cambio de formatos 
# La columna 'posted' se encuentra en formato string y se cambiara a un formato de fechas en una nueva columna llamada 'posted_date'
import re
from datetime import datetime
# Define una función para convertir las fechas
def convertir_a_fecha(fecha_str):
    # Usar una expresión regular para extraer el mes, el día y el año
    match = re.search(r'(\w+) (\d+), (\d+)', fecha_str)
    if match:
        mes_str, dia, año = match.groups()
        mes = datetime.strptime(mes_str, "%B").month  # Convertir el nombre del mes a número
        fecha = datetime(int(año), mes, int(dia))
        return fecha
    else:
        return None

# Aplica la función para convertir la columna 'posted_date' a formato de fecha
df_reviews['posted_date'] = df_reviews['posted'].apply(convertir_a_fecha)

# Eliminar los valores que no tienen un año especifico por lo que se asumiran como datos nulos
df_reviews = df_reviews.dropna(subset=['posted_date'])

# Eliminar las filas con datos nulos de la columna review
df_reviews = df_reviews.dropna(subset=['review'])

# Eliminar las filas duplicadas
df_reviews = df_reviews.drop_duplicates(keep='first')

# Convertir los datos de la columna 'item_id' a formato entero
df_reviews['item_id'] = df_reviews['item_id'].astype(int)

#Eliminar la columna 'posted' que ya no sera utilizada y las columnas 'funny' y 'last edited' que tienen muchos nulos y no serviran para el analisis
df_reviews = df_reviews.drop('posted', axis=1)
df_reviews = df_reviews.drop('funny', axis=1)
df_reviews = df_reviews.drop('last_edited', axis=1)

In [16]:
df_reviews["posted_date"] = pd.to_datetime(df_reviews["posted_date"])
df_reviews["year_posted"] = df_reviews["posted_date"].dt.year
df_UserRecommend = df_reviews[['item_id',"recommend",'year_posted']]
df_aux= df_games[["item_id","title"]]
df_UserRecommend = pd.merge(df_UserRecommend,df_aux, on="item_id", how="inner")
df_UserRecommend = df_UserRecommend[['title','year_posted',"recommend"]]
df_UserRecommend.to_parquet(r'df_UserRecommend.parquet',index=False)

In [17]:
def UsersRecommend(año: int):
    
    df = df_UserRecommend[(df_UserRecommend['year_posted'] == año) & (df_UserRecommend['recommend'] == True)]
    # Contar el número de recomendaciones para cada juego
    top_juegos = df['title'].value_counts().head(3)
    # Crear el formato de salida
    resultado_final = [{"Puesto {}: {}".format(i+1, juego): recomendaciones} for i, (juego, recomendaciones) in enumerate(top_juegos.items())]
    return resultado_final

# Ejemplo de uso
UsersRecommend(2012)

[{'Puesto 1: Terraria': 168},
 {"Puesto 2: Garry's Mod": 76},
 {'Puesto 3: Portal 2': 58}]

# UsersNotRecommend
Devuelve el top de 3 juegos menos recomendados por usuarios para el año dado.

In [18]:
df_reviews.head(5)
df_UserWorstDeveloper = df_reviews[["item_id","recommend","year_posted"]]
df_info = df_games[["item_id","developer"]]
df_UserWorstDeveloper = pd.merge(df_UserWorstDeveloper,df_info, on ="item_id",how= "inner")
df_UserWorstDeveloper = df_UserWorstDeveloper[(df_UserWorstDeveloper['recommend'] == False)]
df_UserWorstDeveloper = df_UserWorstDeveloper[["developer", "year_posted"]]
df_UserWorstDeveloper.to_parquet(r'df_UserWorstDeveloper.parquet',index=False)

In [19]:
def UserWorstDeveloper(year):
    # Filtrar el DataFrame por el año especificado
    df_filtered = df_UserWorstDeveloper[df_UserWorstDeveloper['year_posted'] == year]
    # Contar la cantidad de publicaciones por desarrollador
    developer_counts = df_filtered['developer'].value_counts()
    # Tomar los primeros N resultados
    top_developers = developer_counts.head(3)
    # Crear la lista con el formato solicitado
    result_list = [{"Puesto {}: {}".format(i + 1, developer): count} for i, (developer, count) in enumerate(top_developers.items())]

    return result_list
UserWorstDeveloper(2012)

[{'Puesto 1: Coreplay GmbH,Bigmoon Studios': 3},
 {'Puesto 2: OVERKILL Software': 1},
 {'Puesto 3: Slant Six Games': 1}]

# Sentiment_analysis
Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.

In [20]:
df_10 = df_reviews[["item_id","sentiment_analysis"]]
df_11 = df_games[["developer","item_id"]]
df_Sentiment_Analysis = pd.merge(df_10,df_11, on = "item_id", how= "inner") 
df_Sentiment_Analysis = df_Sentiment_Analysis.groupby(['developer', 'sentiment_analysis']).size().unstack(fill_value=0)
df_Sentiment_Analysis = df_Sentiment_Analysis.reset_index(drop=False)
nuevos_nombres = ["Developer",'Negativo', 'Neutro',"Positivo"]
df_Sentiment_Analysis.columns = nuevos_nombres
df_Sentiment_Analysis["Developer"] = df_Sentiment_Analysis["Developer"].astype(str).str.lower()
df_Sentiment_Analysis.to_parquet("df_Sentiment_Analysis.parquet", index=False )

In [21]:
def Sentiment_analysis(desarrollador):
    # Filtra el DataFrame para obtener la fila correspondiente al desarrollador
    desarrollador =desarrollador.lower()
    desarrollador_fila = df_Sentiment_Analysis[df_Sentiment_Analysis['Developer'] == desarrollador]
    # Extrae los valores de las columnas Negativo, Neutro y Positivo
    negativo = desarrollador_fila['Negativo'].values[0]
    neutro = desarrollador_fila['Neutro'].values[0]
    positivo = desarrollador_fila['Positivo'].values[0]
    # Crea un diccionario con la información en el formato solicitado
    resultado = {
        desarrollador: {
            'Negative': negativo,
            'Neutral': neutro,
            'Positive': positivo
        }
    }

    return resultado
Sentiment_analysis('11 bit studios')

{'11 bit studios': {'Negative': 57, 'Neutral': 24, 'Positive': 57}}

# Machine_Learning

In [12]:
import pandas as pd
# Arreglo de los dataframes para el sistema de recomendacion 
df_reviews = pd.read_parquet("reviews_limpio.parquet")
df_games = pd.read_parquet("games_limpio.parquet")
df_games.rename(columns={'id': 'item_id'}, inplace=True)
df_games=df_games[["item_id","title"]]
df_reviews = df_reviews[["user_id","item_id","recommend"]]
df = pd.merge(df_reviews,df_games, on="item_id", how="inner")
df = df[["item_id","title","recommend"]]
df_filtered = df[df['recommend'] == True]
df = df.astype(str)
df.to_parquet("df_ML.parquet", index= False)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def recomendacion_usuario(user_id :str):
    # Construye una matriz de usuarios y juegos
    user_game_matrix = pd.crosstab(df_reviews['user_id'], df_games['title'])
    try:
        # Encuentra el índice del usuario en la matriz
        user_index = user_game_matrix.index.get_loc(user_id)
    except KeyError:
        print(f"El usuario {user_id} no está presente en los datos.")
        return None
    # Calcula la similitud de coseno entre los usuarios
    cosine_similarities = cosine_similarity(user_game_matrix, user_game_matrix)
    # Obtén las similitudes de coseno para el usuario dado
    similar_users = cosine_similarities[user_index]
    # Encuentra los juegos que el usuario no ha jugado
    games_played = user_game_matrix.loc[user_id]
    unrated_games = games_played[games_played == 0].index
    # Calcula las puntuaciones de recomendación sumando las similitudes de usuarios para los juegos no jugados
    recommendation_scores = user_game_matrix.loc[:, unrated_games].multiply(similar_users, axis=0).sum(axis=0)
    # Ordena las recomendaciones por puntuación descendente
    recommendations = recommendation_scores.sort_values(ascending=False).index.tolist()
    # Limita las recomendaciones a 5 juegos
    top_recommendations = recommendations[:5]

    return top_recommendations

# Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.
usuario_a_recomendar = "76561197970982479"
recomendacion_usuario('76561197970982479')

['弹炸人2222',
 '! That Bastard Is Trying To Steal Our Gold !',
 'ReturnState',
 'Revenge of the Titans',
 'Revenge of the Spirit: Rite of Resurrection']